In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("NIFTY 50.csv")
data.head()

,date,open,high,low,close,volume
0,2015-01-09 09:15:00,8285.45,8295.90,8285.45,8292.10,0
1,2015-01-09 09:16:00,8292.60,8293.60,8287.20,8288.15,0
2,2015-01-09 09:17:00,8287.40,8293.90,8287.40,8293.90,0
3,2015-01-09 09:18:00,8294.25,8300.65,8293.90,8300.65,0
4,2015-01-09 09:19:00,8300.60,8301.30,8298.75,8301.20,0


In [3]:
data['date'] = pd.to_datetime(data['date'])
data = data.set_index('date')

data['open'] = data['open'].resample('3Min').first()
data['high'] = data['high'].resample('3Min').max()
data['low'] = data['low'].resample('3Min').min()
data['close'] = data['close'].resample('3Min').last()
data = data.dropna()

data = data.drop('volume', axis=1)
data['bar_marker'] = pd.Series()
data['sph_spl_marker'] = pd.Series()
data['place_order_marker'] = pd.Series()

data.head()

,open,high,low,close,bar_marker,sph_spl_marker,place_order_marker
date,,,,,,,
2015-01-09 09:15:00,8285.45,8295.90,8285.45,8293.90,NaN,NaN,NaN
2015-01-09 09:18:00,8294.25,8303.00,8293.90,8300.00,NaN,NaN,NaN
2015-01-09 09:21:00,8300.65,8302.90,8293.25,8294.70,NaN,NaN,NaN
2015-01-09 09:24:00,8295.20,8302.55,8289.45,8289.45,NaN,NaN,NaN
2015-01-09 09:27:00,8289.65,8294.15,8286.80,8294.15,NaN,NaN,NaN


In [4]:
def sph_spl_marker(data_per_day):
    searching_for = ''
    place_order_lst = []
    L_begin_index = -1
    L_index = -1
    H_begin_index = -1
    H_index = -1
    
    for i in range(0, len(data_per_day)):
#         print(data_per_day)
#         input("press enter to continue")
        lowest_index = -1
        highest_index = -1
        ab_index = data_per_day.index.get_loc(data_per_day.loc[data_per_day['bar_marker'] == 'AB'].tail(1).index[0])

        if searching_for == '':  
            for prev in range(ab_index, i):
                if data_per_day.loc[data_per_day.index[prev], 'low'].item() > data_per_day.loc[data_per_day.index[i], 'low'].item() and data_per_day.loc[data_per_day.index[prev], 'close'].item() > data_per_day.loc[data_per_day.index[i], 'close'].item():
                    data_per_day.loc[data_per_day.index[i], 'bar_marker'] = 'B1H'
                    searching_for = 'B2H'
                    if lowest_index == -1 or data_per_day.loc[data_per_day.index[prev], 'low'].item() < data_per_day.loc[data_per_day.index[lowest_index], 'low'].item():
                        lowest_index = prev   
                    data_per_day.loc[data_per_day['bar_marker'] == 'AB', 'bar_marker'] = 'AB_old'
                    data_per_day.loc[data_per_day.index[lowest_index], 'bar_marker'] = 'AB'
                elif data_per_day.loc[data_per_day.index[prev], 'high'].item() < data_per_day.loc[data_per_day.index[i], 'high'].item() and data_per_day.loc[data_per_day.index[prev], 'close'].item() < data_per_day.loc[data_per_day.index[i], 'close'].item():
                    data_per_day.loc[data_per_day.index[i], 'bar_marker'] = 'B1L'
                    searching_for = 'B2L'
                    if highest_index == -1 or data_per_day.loc[data_per_day.index[prev], 'high'].item() > data_per_day.loc[data_per_day.index[highest_index], 'high'].item():
                        highest_index = prev
                    data_per_day.loc[data_per_day['bar_marker'] == 'AB', 'bar_marker'] = 'AB_old'
                    data_per_day.loc[data_per_day.index[highest_index], 'bar_marker'] = 'AB'
            continue

        if searching_for == 'B1H':
            for prev in range(ab_index, i):
                if data_per_day.loc[data_per_day.index[prev], 'low'].item() > data_per_day.loc[data_per_day.index[i], 'low'].item() and data_per_day.loc[data_per_day.index[prev], 'close'].item() > data_per_day.loc[data_per_day.index[i], 'close'].item():
                    data_per_day.loc[data_per_day.index[i], 'bar_marker'] = 'B1H'
                    searching_for = 'B2H'
                    if lowest_index == -1 or data_per_day.loc[data_per_day.index[prev], 'low'].item() < data_per_day.loc[data_per_day.index[lowest_index], 'low'].item():
                        lowest_index = prev   
                    data_per_day.loc[data_per_day['bar_marker'] == 'AB', 'bar_marker'] = 'AB_old'
                    data_per_day.loc[data_per_day.index[lowest_index], 'bar_marker'] = 'AB'
            continue

        if searching_for == 'B2H':
            if data_per_day.loc[data_per_day['bar_marker'] == 'AB', 'low'].item() > data_per_day.loc[data_per_day.index[i], 'low'].item() and data_per_day.loc[data_per_day['bar_marker'] == 'AB', 'close'].item() > data_per_day.loc[data_per_day.index[i], 'close'].item():
                data_per_day.loc[data_per_day.index[i], 'bar_marker'] = 'B2H'        
                if len(data_per_day.loc[data_per_day['sph_spl_marker'] == 'L']) == 0:
                    L_index = data_per_day.index.get_loc(data_per_day.iloc[0:i].loc[data_per_day['high'] == data_per_day.iloc[0:i]['high'].max(), 'sph_spl_marker'].index[0])
                    data_per_day.loc[data_per_day.index[L_index], 'sph_spl_marker'] = 'H'
                else:
                    L_begin_index = data_per_day.index.get_loc(data_per_day.loc[data_per_day['sph_spl_marker'] == 'L'].tail(1).index[0])
                    L_index = data_per_day.index.get_loc(data_per_day.iloc[L_begin_index:i].loc[data_per_day['high'] == data_per_day.iloc[L_begin_index:i]['high'].max(), 'sph_spl_marker'].index[0])
                    data_per_day.loc[data_per_day.index[L_index], 'sph_spl_marker'] = 'H'

                place_order_lst.append('H')
                if len(place_order_lst) == 3:
                    if place_order_lst == ['H','L','H']:
                        data_per_day.loc[data_per_day.index[L_index], 'place_order_marker'] = 'Yes'
                    del place_order_lst[0]       

                searching_for = 'B1L'
                data_per_day.loc[data_per_day['bar_marker'] == 'B1H', 'bar_marker'] = 'B1H_old'
                data_per_day.loc[data_per_day['bar_marker'] == 'AB', 'bar_marker'] = 'AB_old'
                data_per_day.loc[data_per_day['bar_marker'] == 'B2H', 'bar_marker'] = 'AB'
            else:
                for prev in range(ab_index, i):
                    if data_per_day.loc[data_per_day.index[prev], 'low'].item() > data_per_day.loc[data_per_day.index[i], 'low'].item() and data_per_day.loc[data_per_day.index[prev], 'close'].item() > data_per_day.loc[data_per_day.index[i], 'close'].item():
                        data_per_day.loc[data_per_day.index[i], 'bar_marker'] = 'B1H'
                        searching_for = 'B2H'
                        if lowest_index == -1 or data_per_day.loc[data_per_day.index[prev], 'low'].item() < data_per_day.loc[data_per_day.index[lowest_index], 'low'].item():
                            lowest_index = prev   
                        data_per_day.loc[data_per_day['bar_marker'] == 'AB', 'bar_marker'] = 'AB_old'
                        data_per_day.loc[data_per_day.index[lowest_index], 'bar_marker'] = 'AB'
            continue

        if searching_for == 'B1L':
            for prev in range(ab_index, i):
                if data_per_day.loc[data_per_day.index[prev], 'high'].item() < data_per_day.loc[data_per_day.index[i], 'high'].item() and data_per_day.loc[data_per_day.index[prev], 'close'].item() < data_per_day.loc[data_per_day.index[i], 'close'].item():
                    data_per_day.loc[data_per_day.index[i], 'bar_marker'] = 'B1L'
                    searching_for = 'B2L'
                    if highest_index == -1 or data_per_day.loc[data_per_day.index[prev], 'high'].item() > data_per_day.loc[data_per_day.index[highest_index], 'high'].item():
                        highest_index = prev
                    data_per_day.loc[data_per_day['bar_marker'] == 'AB', 'bar_marker'] = 'AB_old'
                    data_per_day.loc[data_per_day.index[highest_index], 'bar_marker'] = 'AB'
            continue

        if searching_for == 'B2L':
            if data_per_day.loc[data_per_day['bar_marker'] == 'AB', 'high'].item() < data_per_day.loc[data_per_day.index[i], 'high'].item() and data_per_day.loc[data_per_day['bar_marker'] == 'AB', 'close'].item() < data_per_day.loc[data_per_day.index[i], 'close'].item():
                data_per_day.loc[data_per_day.index[i], 'bar_marker'] = 'B2L'        
                if len(data_per_day.loc[data_per_day['sph_spl_marker'] == 'H']) == 0:
                    H_index = data_per_day.index.get_loc(data_per_day.iloc[0:i].loc[data_per_day['low'] == data_per_day.iloc[0:i]['low'].min(), 'sph_spl_marker'].index[0])
                    data_per_day.loc[data_per_day.index[H_index], 'sph_spl_marker'] = 'L'
                else:
                    H_begin_index = data_per_day.index.get_loc(data_per_day.loc[data_per_day['sph_spl_marker'] == 'H'].tail(1).index[0])
                    H_index = data_per_day.index.get_loc(data_per_day.iloc[H_begin_index:i].loc[data_per_day['low'] == data_per_day.iloc[H_begin_index:i]['low'].min(), 'sph_spl_marker'].index[0])
                    data_per_day.loc[data_per_day.index[H_index], 'sph_spl_marker'] = 'L'  

                place_order_lst.append('L')
                if len(place_order_lst) == 3:
                    if place_order_lst == ['L','H','L']:
                        data_per_day.loc[data_per_day.index[H_index], 'place_order_marker'] = 'Yes'
                    del place_order_lst[0]

                searching_for = 'B1H'
                data_per_day.loc[data_per_day['bar_marker'] == 'B1L', 'bar_marker'] = 'B1L_old'
                data_per_day.loc[data_per_day['bar_marker'] == 'AB', 'bar_marker'] = 'AB_old'
                data_per_day.loc[data_per_day['bar_marker'] == 'B2L', 'bar_marker'] = 'AB'

            else:
                for prev in range(ab_index, i):
                    if data_per_day.loc[data_per_day.index[prev], 'high'].item() < data_per_day.loc[data_per_day.index[i], 'high'].item() and data_per_day.loc[data_per_day.index[prev], 'close'].item() < data_per_day.loc[data_per_day.index[i], 'close'].item():
                        data_per_day.loc[data_per_day.index[i], 'bar_marker'] = 'B1L'
                        searching_for = 'B2L'
                        if highest_index == -1 or data_per_day.loc[data_per_day.index[prev], 'high'].item() > data_per_day.loc[data_per_day.index[highest_index], 'high'].item():
                            highest_index = prev
                        data_per_day.loc[data_per_day['bar_marker'] == 'AB', 'bar_marker'] = 'AB_old'
                        data_per_day.loc[data_per_day.index[highest_index], 'bar_marker'] = 'AB'
            continue
    return data_per_day

In [5]:
data_out = pd.DataFrame()
for idx,data_per_day in data.iloc[0:250].groupby(data.iloc[0:250].index.date):
    data_per_day.loc[data_per_day.index[0],'bar_marker'] = 'AB' 
    data_per_day_updated = sph_spl_marker(data_per_day)
    data_out = data_out.append(data_per_day_updated)

C:\Users\553314\AppData\Local\Continuum\anaconda3\envs\BI_OPS_ML\lib\site-packages\pandas\core\indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
C:\Users\553314\AppData\Local\Continuum\anaconda3\envs\BI_OPS_ML\lib\site-packages\ipykernel_launcher.py:89: FutureWarning: `item` has been deprecated and will be removed in a future version
C:\Users\553314\AppData\Local\Continuum\anaconda3\envs\BI_OPS_ML\lib\site-packages\pandas\core\ops\__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)
C:\Users\553314\AppData\Local\Continuum\anaconda3\envs\BI_OPS_ML\lib\site-packages\ipykernel_launcher.py:46: FutureWarning: `

In [6]:
#data_out.iloc[-30:]
data_out.iloc[120:150]

,open,high,low,close,bar_marker,sph_spl_marker,place_order_marker
date,,,,,,,
2015-01-09 15:15:00,8286.30,8287.55,8282.35,8283.35,B1H_old,NaN,NaN
2015-01-09 15:18:00,8283.10,8283.80,8279.40,8281.75,AB_old,L,Yes
2015-01-09 15:21:00,8282.35,8286.70,8281.75,8286.55,B1L_old,NaN,NaN
2015-01-09 15:24:00,8286.90,8288.05,8282.40,8282.40,AB,NaN,NaN
2015-01-09 15:27:00,8282.35,8284.25,8278.05,8280.25,B1H,NaN,NaN
2015-01-12 09:15:00,8291.35,8291.35,8245.60,8257.85,AB_old,L,NaN
2015-01-12 09:18:00,8258.15,8270.60,8257.85,8267.45,AB_old,NaN,NaN
2015-01-12 09:21:00,8266.05,8274.45,8263.65,8265.80,NaN,NaN,NaN
2015-01-12 09:24:00,8266.75,8266.75,8254.45,8255.90,B1H_old,NaN,NaN


In [7]:
import mplfinance as mpf
import matplotlib.pyplot as plt

data_plt = data_out.copy()
data_plt.columns = ['Open', 'High', 'Low', 'Close', 'Bar_Marker', 'Sph_Spl_Marker', 'Place_Order_Marker']
mpf.plot(data_plt.iloc[120:150][['Open', 'High', 'Low', 'Close']], type='candlestick')

plt.show();

<Figure size 800x575 with 2 Axes>